Itility Assignment Yiannis Makantasis

Questions:

• Can you analyze the measurement data and give advice on which IPC’s could be downsized?

• Describe the steps that you have taken to get to the advice (please mention which data you have included and why).

• Do a general check on the data and outline where you see strange activity (and why).

• What else would be needed to run such an analysis continuously across all IPCs?

In [ ]:
# Import all the packages needed

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Specify the path to the CSV file
csv_path_file = "C:/Users/User/Desktop/Itility Test Case/new_dataset.csv"

# Read the CSV file uing a pandas DataFrame
data = pd.read_csv(csv_path_file)

# print the DataFrame for information
# print(df)

In [ ]:
# Count the number of distinct values in the specified column
distinct_ipc_values = data['IPC'].nunique()

# Print the number of distinct values in the column
print("Number of distinct IPC values:", distinct_ipc_values)

In [ ]:
# Print general information regarding the DataFrame
data.info()

In [ ]:
# Print the first 5 rows of the DataFrame to get an idea of what the dataset looks like
data.head()

In [ ]:
# Check if there are any duplicate entries
dup_rows = data[data.duplicated()]
print("The number of duplicated rows is", dup_rows)

In [ ]:
# This cell willl first find the cells in CpuMHz that have two numbers inside the cell instead of one, and will remove them.
# The second step will be to turn the Dtype of this column from object to int-64 to be used for the later calculations

# Define a custom function to check for two values separated by a space
def has_two_values(cell):
    if isinstance(cell, str) and ' ' in cell:
        values = cell.split()
        if len(values) == 2 and all(value.isdigit() for value in values):
            return True
    return False

# Apply the custom function to the 'CPU' column
cpu_contains_two_values = data[data['CpuMHz'].apply(has_two_values)]

# Print the rows where 'CPU' contains two objects
print(cpu_contains_two_values)

data.drop(data.loc[data['CpuMHz'].str.contains(' ') == True].index, inplace=True)
data['CpuMHz'] = data['CpuMHz'].astype(np.int64)

In [ ]:
# Print the new general information to also check the changed Dtype of CpuMHz from onject to int-64
data.info()

In [ ]:
# Similarly to CpuMHz, two numbers inside cells were found in the column MemoryMB. The 70 rows containing these cells are then removed
# The types of both MemoryMB and NumCpu are then alsso changed to int-64

# Define a custom function to check for two values separated by a space
def has_two_values(cell):
    if isinstance(cell, str) and ' ' in cell:
        values = cell.split()
        if len(values) == 2 and all(value.isdigit() for value in values):
            return True
    return False

# Apply the custom function to the 'CPU' column
memorymb_contains_two_values = data[data['MemoryMB'].apply(has_two_values)]

# Print the rows where 'CPU' contains two objects
print(memorymb_contains_two_values)

data.drop(data.loc[data['MemoryMB'].str.contains(' ') == True].index, inplace=True)
data['MemoryMB'] = data['MemoryMB'].astype(np.int64)
data['NumCpu'] = data['NumCpu'].astype(np.int64)



In [ ]:
# Print the new general information to also check the changed Dtype of MemoryMB and NumCpu from onject to int-64
data.info()

In [ ]:
data.loc[(data['MetricId'] == 'MemActive')].describe()

In [ ]:
data.loc[(data['MetricId'] == 'CpuUsageMHz')].describe()

In [ ]:
boxplot = data.loc[(data['MetricId'] == 'CpuUsageMHz')].boxplot(column=['MaxValue'], vert=False)

In [ ]:
plt.show()

In [ ]:
sns.countplot(x = data.loc[(data['AvgValue'] > 0.8 * data['CpuMHz']) & (data['MetricId'] == 'CpuUsageMHz')].groupby(['IPC']).size())
plt.xlabel("Number of times upper limit has been surpassed", size = 14, labelpad=14)
plt.ylabel("Number of IPC's", size=14, labelpad=14)
plt.tick_params(axis='x', labelsize=6)
plt.tick_params(axis='y', labelsize=13)

plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [6]:
cpuData = data.loc[(data['MetricId'] == 'CpuUsageMHz' )].copy()
cpuData.head()

,IPC,Data Factory,time,AvgValue,MinValue,MaxValue,MetricId,MemoryMB,CpuMHz,NumCpu
1,ITLT4301,1,01/05/2021,9894437496,2934354523,6730675687,CpuUsageMHz,0,5600,2
3,ITLT4301,1,02/05/2021,1146709829,3026053102,840875968,CpuUsageMHz,0,5600,2
5,ITLT4301,1,03/05/2021,1210612276,3209450259,9068989447,CpuUsageMHz,0,5600,2
7,ITLT4301,1,04/05/2021,119882602,3392847417,6904902987,CpuUsageMHz,0,5600,2
9,ITLT4301,1,05/05/2021,1252555738,3576244575,728086716,CpuUsageMHz,0,5600,2


In [11]:
for index, row in cpuData.iterrows():
    max_possible_cpu_utilization = row['NumCpu'] * row['CpuMHz']
    actual_cpu_utilization = (row['MaxValue'] / max_possible_cpu_utilization) * 100
    
    if actual_cpu_utilization < 70:
        print(f"IPC {index} is using less than 20% of its CPU capacity.")

IPC 67819 is using less than 20% of its CPU capacity.
IPC 157569 is using less than 20% of its CPU capacity.
IPC 294805 is using less than 20% of its CPU capacity.


In [ ]:
cpuData['Oversized'] = 0
cpuData.loc[cpuData.MaxValue <= 0.5 * cpuData.CpuMHz ,'Oversized']= 1

In [ ]:
Oversized = cpuData.groupby(['IPC']).agg({'Oversized': ['count', 'sum']})
Oversized.columns = ['IPC', 'Oversized']
Oversized['RecommendForDownscale'] = 0
Oversized.loc[Oversized.IPC == Oversized.Oversized, 'RecommendForDownscale'] = 1
Oversized

In [ ]:
Oversized['RecommendForDownscale'].to_csv('C:/Users/User/Desktop/Itility Test Case/RecommendForDownscale.csv')